In [ ]:
#!pip install openai

In [ ]:
# OpenAI Settings
APIKey = ''

import openai
openai.api_key = APIKey

In [14]:
#declare connection with OpenAI
from openai import OpenAI
client = OpenAI(
    api_key=APIKey,
)

## Queries

In [15]:
# Function to create a file with the Files API
def create_file(file_path):
  with open(file_path, "rb") as file_content:
    result = client.files.create(
        file=file_content,
        purpose="vision",
    )
    return result.id

In [16]:
system_prompt = """Jesteś automatem analizującym obrazy.   """

In [17]:
user_prompt = """
W dostarczonej treści zidentyfikuj i wypisz produkty oraz ich ceny """

In [18]:
# Ścieżka do obrazu
image_path = "./Zrzut-allegro.png"
# Pobieranie fileID
file_id = create_file(image_path)

In [19]:
from pydantic import BaseModel
from typing import List
import json

class Product(BaseModel):
    name: str
    price: float

In [20]:
msg=[
    {"role": "developer", "content": system_prompt}, 
    {
      "role": "user",
      "content": [
        {
          "type": "input_text",
          "text": user_prompt,
        },
        {
          "type": "input_image",
          "file_id": file_id
          },
      ],
    }
  ]

In [21]:
# Wysłanie obrazu do modelu OpenAI
response = client.responses.parse(
    input=msg,
    model="gpt-5-mini",
    text_format=Product
)
resp = response.output_text.strip() 
print(resp)

{"name":"Tyranids Battleforce Crusher Stampede 2026 - 40k Maleceptor Haruspex","price":825}


In [22]:
import json

data = json.loads(resp)
print(f"Cena brutto: {data['price']:.2f} zł")
print(f"Cena netto: {data['price'] / 1.23:.2f} zł")


Cena brutto: 825.00 zł
Cena netto: 670.73 zł


In [23]:
system_prompt = """Jesteś automatem analizującym obrazy.   
"""
user_prompt = """
W dostarczonej treści zidentyfikuj i wypisz produkty oraz ich ceny """

In [25]:
# Ścieżka do obrazu
image_path = "./zrzut-aos.png"
# Pobieranie fileID
file_id = create_file(image_path)

In [26]:
from pydantic import BaseModel
from typing import List
import json

class Product(BaseModel):
    name: str
    capacity: str
    price: float
    rating: float


class ProductsResponse(BaseModel):
    products: List[Product]

In [27]:
msg=[
    {"role": "developer", "content": system_prompt}, 
    {
      "role": "user",
      "content": [
        {
          "type": "input_text",
          "text": user_prompt,
        },
        {
          "type": "input_image",
          "file_id": file_id
          },
      ],
    }
  ]

In [28]:
# Wysłanie obrazu do modelu OpenAI
response = client.responses.parse(
    input=msg,
    model="gpt-5-mini",
    text_format=ProductsResponse
)
print(response.output_text.strip())

{
  "products": [
    {
      "name": "Scinari Enlightener",
      "capacity": "",
      "price": 121.0,
      "rating": 0
    },
    {
      "name": "Hurakan Spirit of the Wind",
      "capacity": "",
      "price": 215.0,
      "rating": 0
    },
    {
      "name": "Vanari Bannerblade",
      "capacity": "",
      "price": 140.0,
      "rating": 0
    },
    {
      "name": "Hurakan Windchargers",
      "capacity": "",
      "price": 245.0,
      "rating": 0
    },
    {
      "name": "Hurakan Windmage",
      "capacity": "",
      "price": 152.5,
      "rating": 0
    },
    {
      "name": "Sevireth, Lord of the Seventh Wind",
      "capacity": "",
      "price": 215.0,
      "rating": 0
    },
    {
      "name": "Scinari Calligrave",
      "capacity": "",
      "price": 121.0,
      "rating": 0
    },
    {
      "name": "Vanari Lord Regent",
      "capacity": "",
      "price": 215.0,
      "rating": 0
    }
  ]
}


In [29]:
import pandas as pd
data = json.loads(response.output_text.strip())

# Zamiana dict → DataFrame
df = pd.DataFrame(data["products"])

# Wyświetlenie
df

,name,capacity,price,rating
0,Scinari Enlightener,,121.0,0
1,Hurakan Spirit of the Wind,,215.0,0
2,Vanari Bannerblade,,140.0,0
3,Hurakan Windchargers,,245.0,0
4,Hurakan Windmage,,152.5,0
5,"Sevireth, Lord of the Seventh Wind",,215.0,0
6,Scinari Calligrave,,121.0,0
7,Vanari Lord Regent,,215.0,0
